<a href="https://colab.research.google.com/github/jojosam90/ltsm/blob/main/Multivariate%20Time%20Series%20Modeling%20using%20LSTM_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
from google.colab import drive
drive.mount('/content/drive')

#https://www.kaggle.com/code/francoisraucent/forecasting-electricity-consumption-of-germany

#https://towardsdatascience.com/forecasting-the-future-power-consumption-of-germany-using-lstm-rnn-and-dnn-d8e05e7fdc0a

#https://www.kaggle.com/code/ashfakyeafi/optimization-of-ev-charging

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
# import pandas as pd
# from pandas import DataFrame
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import legend
# from sklearn.model_selection import train_test_split
# from keras.preprocessing.sequence import TimeseriesGenerator
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# import tensorflow as tf
# import datetime
# import numpy as np
# import seaborn as sns

# mpl.rcParams['figure.figsize'] = (10,8)
# mpl.rcParams['axes.grid'] = False

In [83]:
import pandas as pd
import numpy as np
import math

import holidays

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler, OneHotEncoder, FunctionTransformer
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (
    train_test_split, KFold, GridSearchCV, ParameterGrid,
)
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, DMatrix, plot_importance
from xgboost import cv as xgb_cv

In [84]:
df = pd.read_csv('/content/drive/MyDrive/dataset/Energy/Year 2017.csv')

In [85]:
df

,_id,CP ID,Connector,Start Date,Start Time,End Date,End Time,Total kWh,Site,Model
0,1,51519,2,2017-08-31T00:00:00,22:12,2017-08-31T00:00:00,23:11,18.270,"Broxden Park & Ride, Perth",APT Triple Rapid Charger
1,2,51250,1,2017-08-31T00:00:00,22:01,2017-08-31T00:00:00,22:31,19.450,"Crown Inn Wynd Car Park, Auchterarder",Siemens Triple Rapid Charger
2,3,50575,2,2017-08-31T00:00:00,21:21,2017-08-31T00:00:00,21:59,19.710,"South Inch Car Park, Perth",APT Triple Rapid Charger
3,4,51516,2,2017-08-31T00:00:00,20:43,2017-08-31T00:00:00,21:03,9.840,Kinross Park and Ride,APT Triple Rapid Charger
4,5,50994,2,2017-08-31T00:00:00,19:50,2017-08-31T00:00:00,20:50,14.170,"Leslie Street Car Park, Blairgowrie",APT Triple Rapid Charger
...,...,...,...,...,...,...,...,...,...,...
13671,13672,50745,2,2016-01-09T00:00:00,09:22,2016-01-09T00:00:00,09:37,3.580,Kinross Park and Ride,APT Triple Rapid Charger
13672,13673,50281,1,2016-01-09T00:00:00,08:54,2016-01-09T00:00:00,16:37,10.380,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet
13673,13674,50285,1,2016-01-09T00:00:00,08:22,2016-01-09T00:00:00,14:32,13.930,Broxden Park & Ride,APT 22kW Dual Outlet
13674,13675,50281,2,2016-01-09T00:00:00,07:51,2016-01-09T00:00:00,09:01,3.870,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet


In [86]:
# drop NA values
df = df.dropna()

In [87]:
# delete duplicate rows
df = df.drop_duplicates()

In [88]:
df['StartDateTime'] = pd.to_datetime(df['Start Date'].astype(str) + ' ' +df['Start Time'].astype(str))
df['EndDateTime'] = pd.to_datetime(df['End Date'].astype(str) + ' ' +df['End Time'].astype(str))

In [89]:
df

,_id,CP ID,Connector,Start Date,Start Time,End Date,End Time,Total kWh,Site,Model,StartDateTime,EndDateTime
0,1,51519,2,2017-08-31T00:00:00,22:12,2017-08-31T00:00:00,23:11,18.270,"Broxden Park & Ride, Perth",APT Triple Rapid Charger,2017-08-31 22:12:00,2017-08-31 23:11:00
1,2,51250,1,2017-08-31T00:00:00,22:01,2017-08-31T00:00:00,22:31,19.450,"Crown Inn Wynd Car Park, Auchterarder",Siemens Triple Rapid Charger,2017-08-31 22:01:00,2017-08-31 22:31:00
2,3,50575,2,2017-08-31T00:00:00,21:21,2017-08-31T00:00:00,21:59,19.710,"South Inch Car Park, Perth",APT Triple Rapid Charger,2017-08-31 21:21:00,2017-08-31 21:59:00
3,4,51516,2,2017-08-31T00:00:00,20:43,2017-08-31T00:00:00,21:03,9.840,Kinross Park and Ride,APT Triple Rapid Charger,2017-08-31 20:43:00,2017-08-31 21:03:00
4,5,50994,2,2017-08-31T00:00:00,19:50,2017-08-31T00:00:00,20:50,14.170,"Leslie Street Car Park, Blairgowrie",APT Triple Rapid Charger,2017-08-31 19:50:00,2017-08-31 20:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...
13671,13672,50745,2,2016-01-09T00:00:00,09:22,2016-01-09T00:00:00,09:37,3.580,Kinross Park and Ride,APT Triple Rapid Charger,2016-01-09 09:22:00,2016-01-09 09:37:00
13672,13673,50281,1,2016-01-09T00:00:00,08:54,2016-01-09T00:00:00,16:37,10.380,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 08:54:00,2016-01-09 16:37:00
13673,13674,50285,1,2016-01-09T00:00:00,08:22,2016-01-09T00:00:00,14:32,13.930,Broxden Park & Ride,APT 22kW Dual Outlet,2016-01-09 08:22:00,2016-01-09 14:32:00
13674,13675,50281,2,2016-01-09T00:00:00,07:51,2016-01-09T00:00:00,09:01,3.870,"Rie-Achan Road Car Park, Pitlochry",APT 22kW Dual Outlet,2016-01-09 07:51:00,2016-01-09 09:01:00


In [90]:
df = df.drop(columns=['_id','CP ID','Connector','Start Date','Start Time','End Date','End Time','Site','Model'])
df=df[['StartDateTime','EndDateTime','Total kWh']]

In [91]:
df["StartDateTime"] = pd.to_datetime(df["StartDateTime"])

df = df.sort_values(by="StartDateTime")

In [92]:
df

,StartDateTime,EndDateTime,Total kWh
13675,2016-01-09 07:21:00,2016-01-09 07:27:00,2.084
13674,2016-01-09 07:51:00,2016-01-09 09:01:00,3.870
13673,2016-01-09 08:22:00,2016-01-09 14:32:00,13.930
13672,2016-01-09 08:54:00,2016-01-09 16:37:00,10.380
13671,2016-01-09 09:22:00,2016-01-09 09:37:00,3.580
...,...,...,...
949,2017-12-08 19:51:00,2017-12-08 20:34:00,10.097
948,2017-12-08 21:04:00,2017-12-08 21:27:00,13.000
947,2017-12-08 22:11:00,2017-12-08 22:39:00,20.010
946,2017-12-08 22:16:00,2017-12-08 22:55:00,19.700


In [93]:
#df1[~(df1 == 0).all(axis=1)]

#make sure every hour is in the datetime

#df = df.set_index("StartDateTime").resample("15min").sum()

#df

In [94]:
df = df.drop(columns="EndDateTime").set_index("StartDateTime")

In [95]:
df

,Total kWh
StartDateTime,
2016-01-09 07:21:00,2.084
2016-01-09 07:51:00,3.870
2016-01-09 08:22:00,13.930
2016-01-09 08:54:00,10.380
2016-01-09 09:22:00,3.580
...,...
2017-12-08 19:51:00,10.097
2017-12-08 21:04:00,13.000
2017-12-08 22:11:00,20.010


In [96]:
df.index = pd.to_datetime(df.index)

In [97]:
df.index.name = "Time"

In [99]:
df = df.groupby(pd.Grouper(freq="h")).mean()

In [101]:
df

,Total kWh
Time,
2016-01-09 07:00:00,2.977
2016-01-09 08:00:00,12.155
2016-01-09 09:00:00,3.580
2016-01-09 10:00:00,7.105
2016-01-09 11:00:00,21.780
...,...
2017-12-08 19:00:00,7.730
2017-12-08 20:00:00,NaN
2017-12-08 21:00:00,13.000


In [ ]:
STUDY_START_DATE = pd.Timestamp("2015-01-01 00:00", tz="utc")
STUDY_END_DATE = pd.Timestamp("2020-01-31 23:00", tz="utc")